# Домашняя работа часть 2  (5 баллов): совмещение спарсификации и квантизации.


В этом задании вам предстоит совместить спарсификацию и квантизацию.

Что бы упросить задание, мы не будем квантизовать всю модель, а квантизуем только по слоям.

Резальтат работы это метрики L1 и L2  для $|C(W)X^T - WX^T|$ для каждого слоя, $С$  функция спарсификации и квантизации.


В папке llama7b_weights содержатся веса для одоного слоя LLaMа7b и сопутствующие активации в папке llama7b_act_scales (аггрегировые по датасету)

In [20]:
import os
import torch 

In [112]:
# load weights

FOLDER = "../llama7b_weights/"
names = os.listdir(FOLDER)
weight_paths =  {name.replace(".pt",'') :os.path.join(FOLDER, name) for name in names}
names = [name.replace(".pt",'') for name in names]
W = torch.load(weight_paths[names[1]])
W.shape

torch.Size([4096, 11008])

In [113]:
# Расчет матрицы гессе для слоя
X = torch.load("../llama7b_act_scales/Llama-2-7b-hf.pt")[names[1]]

H = torch.outer(X, X)

damp = 0.01 * torch.mean(torch.diag(H))
diag = torch.arange(X.shape[0])
H[diag, diag] += damp

H = torch.linalg.cholesky(H)
H = torch.cholesky_inverse(H)
H = torch.linalg.cholesky(H, upper=True)
Hinv = H
H.shape

torch.Size([11008, 11008])

Для вдохновения рекомендуем воспользоваться кодов QUIK

https://github.com/IST-DASLab/QUIK/blob/9558d7121c698174fc93940e0b52c38c746c97ea/experiments/quik_utils.py#L80

# Ваше решение тут ...

#### Пример расчета ошибки

In [116]:
W = W.float()
X = X.float()

In [121]:
#  Представим что l2 наша функция квантизации

l2 = ((W@X.T - torch.round(W)@X.T)**2).mean()
l2

tensor(158.4233)